# Here, I will analyze the saved study

In [60]:
import joblib
import numpy as np
import tensorflow as tf

from tensorflow import keras
from keras import layers
import optuna

import matplotlib.pyplot as plt

### Setting Up GPU as a training device

In [61]:
physical_devices = tf.config.list_physical_devices()

print(physical_devices)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [62]:
tf.config.set_visible_devices(physical_devices[1], 'GPU')

### Loading Data

In [63]:
N_TRAIN_EXAMPLES = 50000
N_TEST_EXAMPLES = 10000

CLASSES=10

In [64]:
from sklearn.model_selection import train_test_split

(X_train, Y_train), (X_test, Y_test) = keras.datasets.cifar10.load_data()

# Concatenate train and test images
X = np.concatenate((X_train, X_test))
y = np.concatenate((Y_train, Y_test))

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=N_TRAIN_EXAMPLES, test_size=N_TEST_EXAMPLES, random_state=1)

# Getting dummy variables
y_train_fixed = np.zeros((y_train.shape[0], 10))
y_test_fixed = np.zeros((y_test.shape[0], 10))

i = 0
for [val] in y_train:
  y_train_fixed[i][val] = 1
  i += 1
  
i = 0
for [val] in y_test:
  y_test_fixed[i][val] = 1
  i += 1

### Loading study

In [65]:
study = joblib.load('studies/optuna_study_saving_test.pkl')

print(study.best_params)
print(study.best_value)

{'filters_1': 64, 'kernel_size_1': 3, 'filters_2': 64, 'kernel_size_2': 5, 'strides_2': 1, 'max_pool_size_1': 3, 'dense_size_1': 256, 'learning_rate': 8.253374315877225e-05}
0.6603999733924866


### Building model

In [66]:
model = keras.Sequential(
  [
    keras.Input(shape=(32, 32, 3)),
    layers.Conv2D(32, kernel_size=3, padding='same', activation="relu"),
    layers.Dropout(0.1),
    layers.Conv2D(64, kernel_size=3, padding='same', activation="relu"),
    layers.Dropout(0.1),
    layers.Conv2D(128, kernel_size=3, padding='same', activation="relu"),
    layers.Dropout(0.1),
    layers.Conv2D(64, kernel_size=5, strides=2, padding='same', activation="relu"),
    layers.Dropout(0.1),
    layers.MaxPooling2D(pool_size=3),
    layers.Flatten(),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.1),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(CLASSES, activation="softmax"),
  ]
)

model.summary()

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_41 (Conv2D)          (None, 32, 32, 32)        896       
                                                                 
 dropout_73 (Dropout)        (None, 32, 32, 32)        0         
                                                                 
 conv2d_42 (Conv2D)          (None, 32, 32, 64)        18496     
                                                                 
 dropout_74 (Dropout)        (None, 32, 32, 64)        0         
                                                                 
 conv2d_43 (Conv2D)          (None, 32, 32, 128)       73856     
                                                                 
 dropout_75 (Dropout)        (None, 32, 32, 128)       0         
                                                                 
 conv2d_44 (Conv2D)          (None, 16, 16, 64)      

In [67]:
optimizer = keras.optimizers.Adam()
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [68]:
%%time
batch_size = 256
epochs = 30
validation_split = 0.2
history = model.fit(X_train, y_train_fixed, batch_size=batch_size, epochs=epochs, validation_split=validation_split)

Epoch 1/30
 78/157 [=============>................] - ETA: 10s - loss: 4.2102 - accuracy: 0.1012

In [ ]:
# Evaluate the model

loss, score = model.evaluate(X_test, y_test_fixed)

print(loss, score)

### Save model to file

In [ ]:
model.save("models/cifar10_model_1_D.keras")

In [ ]:
# Plotting the loss curve
plt.figure(figsize=[6, 4])
plt.plot(history.history['loss'], 'black', linewidth=2.0)
plt.plot(history.history['val_loss'], 'blue', linewidth=2.0)
plt.legend(['Training Loss', 'Validation Loss'], fontsize=12)
plt.xlabel('Epochs', fontsize=10)
plt.ylabel('Loss', fontsize=10)
plt.title('Loss Curves', fontsize=12)

In [ ]:
# Plotting the accuracy curve
plt.figure(figsize=[6, 4])
plt.plot(history.history['accuracy'], 'black', linewidth=2.0)
plt.plot(history.history['val_accuracy'], 'blue', linewidth=2.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'], fontsize=12)
plt.xlabel('Epochs', fontsize=10)
plt.ylabel('Accuracy', fontsize=10)
plt.title('Accuracy Curves', fontsize=12)